In [1]:
import mlflow
import os
import pandas as pd
import sys

sys.path.append('../')

# Data Collection

- Scrape 20 years of data from pro football reference

In [ ]:
from src.scraper import ProFootballReferenceScraper

scraper = ProFootballReferenceScraper(data_dir="../data")
scraper.scrape_years(start_year=1999, end_year=2024)

# Data Processing

- Combine the stat tables for all years and save them to the silver layer
- Build the gold table from the different stat tables

In [2]:
from src.processor import DataProcessor

processor = DataProcessor(data_dir="../data")
processor.process_all_data()

Processing files matching: *_player_fantasy_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 280.41it/s]
2025-07-30 09:06:44,159 - src.processor - INFO - Saved player_fantasy_stats.csv to ../data/silver/player_fantasy_stats.csv
Processing files matching: *_player_receiving_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 196.19it/s]
2025-07-30 09:06:50,489 - src.processor - INFO - Saved player_receiving_stats.csv to ../data/silver/player_receiving_stats.csv
Processing files matching: *_player_rushing_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 196.45it/s]
2025-07-30 09:06:55,160 - src.processor - INFO - Saved player_rushing_stats.csv to ../data/silver/player_rushing_stats.csv
Processing files matching: *_player_passing_stats.csv:   0%|          | 0/25 [00:00<?, ?it/s]2025-07-30 09:06:55,166 - src.processor - INFO - Added missing pass_qbr column to ../data/bronze/2001_player_passing_stats.csv
2025-07-30 09:06:55,169 - src.processor - INFO - Added missing pass_qbr column to ../data/bro

## Data Analysis 

- Do some data quality checks on the final stats
- Look at which features are the most and least informative

In [2]:
from src.analysis import DataAnalysis

analysis = DataAnalysis()

2025-07-30 09:18:00,700 - src.analysis - INFO - Loaded training data: 9431 rows
2025-07-30 09:18:00,706 - src.analysis - INFO - Loaded live data: 374 rows


In [3]:
analysis.run_training_data_quality_checks()
analysis.run_live_data_quality_checks()

In [ ]:

analysis.generate_feature_analysis()

## Modelling

- Start with linear regression
- Become more advanced
- Forward, Backward, Feature Selection
- Models per target

In [ ]:
from src.modelling import FantasyModel

model = FantasyModel(data_dir="../data", target_col="ppr_fantasy_points_per_game")
# model = FantasyModel(data_dir="../data", target_col="standard_fantasy_points_per_game")

In [ ]:

data = model.split_data()

In [ ]:
model_eval_search = model.run_model_eval(data)

model_eval_results_df = pd.DataFrame(model_eval_search.cv_results_)

display(model_eval_results_df[['param_model', 'mean_test_r2', 'mean_test_rmse', 'std_test_r2']].sort_values(by='mean_test_r2', ascending=False))

In [ ]:
ridge_eval_search = model.run_model_tuning(data, "random_forest")

ridge_eval_results_df = pd.DataFrame(ridge_eval_search.cv_results_)

display(
    ridge_eval_results_df[['param_model__n_estimators', 'mean_test_r2', 'mean_test_rmse', 'std_test_r2']]
    .sort_values(by=['mean_test_r2', 'mean_test_rmse'], ascending=[False, True])
)

In [ ]:
random_forest_eval_search = model.run_model_tuning(data, "random_forest")

random_forest_eval_results_df = pd.DataFrame(random_forest_eval_search.cv_results_)

display(
    ridge_eval_results_df[['param_model__n_estimators', 'param_model__max_depth' 'mean_test_r2', 'mean_test_rmse', 'std_test_r2']]
    .sort_values(by=['mean_test_r2', 'mean_test_rmse'], ascending=[False, True])
)

In [ ]:
svr_eval_search = model.run_model_tuning(data, "svr")

svr_eval_results_df = pd.DataFrame(svr_eval_search.cv_results_)

display(
    ridge_eval_results_df[['param_model__C', 'param_model__kernel', 'param_model__gamma', 'mean_test_r2', 'mean_test_rmse', 'std_test_r2']]
    .sort_values(by=['mean_test_r2', 'mean_test_rmse'], ascending=[False, True])
)

## Final Predictions!

#### First on the test set

In [ ]:
preds_df = model.make_test_predictions(data, "ridge")

view_year = 2024
year_preds = model.view_year_test_predictions(preds_df, view_year)
year_preds.to_csv(os.path.join(model.predictions_dir, f"{model.target_col}_{view_year}_predictions.csv"), index=False)

print(f"Predictions for {model.target_col} in {view_year}:")
print(year_preds.head(10))

#### Now for 2025

In [ ]:
# Make final predictions